## 0. Libarary 불러오기 및 경로설정

In [74]:
import os
from typing import Tuple, List
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset

from torchvision import transforms
from torchvision.transforms import *
import torchvision.models as models
from tqdm.notebook import tqdm

import albumentations
import albumentations.pytorch.transforms

## 1. Train Dataset 정의

### age

In [73]:
class AgeDataset(Dataset):
    def __init__(self, train_path, transform):
        self.transform = transform
        profiles = os.listdir(train_dir)
        indices = []
        for profile in profiles:
            if profile.startswith('.'):
                continue

            id, gender, race, age = profile.split('_')
            gender = 0 if gender == 'male' else 1

            names = os.listdir(os.path.join(train_dir, profile))
            for name in names:
                if name.startswith('.'):
                    continue

                if name.startswith('mask'):
                    mask = 0
                elif name.startswith('in'):
                    mask = 1
                else:
                    mask = 2
                name_path = os.path.join(train_dir, profile, name)
                indices.append((name_path, gender, age, mask))
        ## profiles_df
        # gender: male -> 0, female -> 1
        # mask: mask -> 0, incorrect -> 1, normal -> 2
        profiles_df = pd.DataFrame(indices, columns=['path', 'gender', 'age', 'mask'])
        self.X = profiles_df.path
        self.Y = profiles_df.age
        

    def __getitem__(self, index):
        image = Image.open(self.X[index])
        if self.transform:
            if isinstance(self.transform, transforms.Compose):
                image = self.transform(image)
            else:
                image = self.transform(image=np.array(image))['image']
            
        return image, int(self.Y[index])
    
    def split_dataset(self) -> Tuple[Subset, Subset]:
        n_val = int(len(self.X) * 0.2)
        n_train = len(self.X) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set
        
        
    def __len__(self):
        return len(self.X) 

### gender

In [75]:
class GenderDataset(Dataset):
    def __init__(self, train_path, transform):
        self.transform = transform
        profiles = os.listdir(train_dir)
        indices = []
        for profile in profiles:
            if profile.startswith('.'):
                continue

            id, gender, race, age = profile.split('_')
            gender = 0 if gender == 'male' else 1

            names = os.listdir(os.path.join(train_dir, profile))
            for name in names:
                if name.startswith('.'):
                    continue

                if name.startswith('mask'):
                    mask = 0
                elif name.startswith('in'):
                    mask = 1
                else:
                    mask = 2
                name_path = os.path.join(train_dir, profile, name)
                indices.append((name_path, gender, age, mask))
        ## profiles_df
        # gender: male -> 0, female -> 1
        # mask: mask -> 0, incorrect -> 1, normal -> 2
        profiles_df = pd.DataFrame(indices, columns=['path', 'gender', 'age', 'mask'])
        self.X = profiles_df.path
        self.Y = profiles_df.gender
        

    def __getitem__(self, index):
        image = Image.open(self.X[index])
        if self.transform:
            if isinstance(self.transform, transforms.Compose):
                image = self.transform(image)
            else:
                image = self.transform(image=np.array(image))['image']
            
        return image, int(self.Y[index])
    
    def split_dataset(self) -> Tuple[Subset, Subset]:
        n_val = int(len(self.X) * 0.2)
        n_train = len(self.X) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set
        
        
    def __len__(self):
        return len(self.X) 

### mask

In [77]:
class MaskDataset(Dataset):
    def __init__(self, train_path, transform):
        self.transform = transform
        profiles = os.listdir(train_dir)
        indices = []
        for profile in profiles:
            if profile.startswith('.'):
                continue

            id, gender, race, age = profile.split('_')
            gender = 0 if gender == 'male' else 1

            names = os.listdir(os.path.join(train_dir, profile))
            for name in names:
                if name.startswith('.'):
                    continue

                if name.startswith('mask'):
                    mask = 0
                elif name.startswith('in'):
                    mask = 1
                else:
                    mask = 2
                name_path = os.path.join(train_dir, profile, name)
                indices.append((name_path, gender, age, mask))
        ## profiles_df
        # gender: male -> 0, female -> 1
        # mask: mask -> 0, incorrect -> 1, normal -> 2
        profiles_df = pd.DataFrame(indices, columns=['path', 'gender', 'age', 'mask'])
        self.X = profiles_df.path
        self.Y = profiles_df.mask
        

    def __getitem__(self, index):
        image = Image.open(self.X[index])
        if self.transform:
            if isinstance(self.transform, transforms.Compose):
                image = self.transform(image)
            else:
                image = self.transform(image=np.array(image))['image']
            
        return image, int(self.Y[index])
    
    def split_dataset(self) -> Tuple[Subset, Subset]:
        n_val = int(len(self.X) * 0.2)
        n_train = len(self.X) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set
        
        
    def __len__(self):
        return len(self.X) 

### transform

In [78]:
# --image_size
image_size = (512, 384)

# --transforms
transform_train = albumentations.Compose([
    albumentations.Resize(height=image_size[0], width=image_size[1], always_apply=True),
    albumentations.OneOf([
        albumentations.HorizontalFlip(p=0.9),  # 좌우 반전
        albumentations.VerticalFlip(p=0.9),  # 상하 반전,
        albumentations.Affine(p=0.9),
        albumentations.ShiftScaleRotate(
            shift_limit=0.2,
            scale_limit=0.2,
            rotate_limit=10,
            border_mode=0,
            p=0.9),
    ], p=0),  # p=0, for cutmix
    albumentations.GaussNoise(p=0.4),
    albumentations.OneOf([
        albumentations.MotionBlur(p=0.9),
        albumentations.MedianBlur(blur_limit=3, p=0.9),
        albumentations.Blur(blur_limit=3, p=0.9),
    ], p=1),
    albumentations.OneOf([
        albumentations.HueSaturationValue(p=0.9),
        albumentations.RGBShift(p=0.9),
        albumentations.ChannelShuffle(p=0.9),
        albumentations.ColorJitter(p=0.9),
    ], p=1),

    albumentations.CoarseDropout(max_holes=4,max_height=30,max_width=30,p=0.2),
    albumentations.RandomBrightnessContrast(p=0.4),

    albumentations.Normalize(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2)), 
    albumentations.pytorch.transforms.ToTensorV2(p=1)
    ], p=1)

transform_test = transforms.Compose([
    Resize(image_size, Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

## 2. Train

### train setting

In [79]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


In [80]:
# find bbox for cutmix
def rand_bbox(size, lam):
    W = size[2]  # batch, channel, width, height
    H = size[3]  # batch, channel, width, height
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (size[2] * size[3]))

    return bbx1, bby1, bbx2, bby2, lam

In [81]:
# --basic settings
train_dir = '/opt/ml/input/data/train/images_augmented_1000'
seed_everything(42)  # seed: 42
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# --dataset & augmentation
age_dataset = AgeDataset(train_dir,transform_train)
age_train, age_val = age_dataset.split_dataset()

gender_dataset = GenderDataset(train_dir,transform_train)
gender_train, gender_val = age_dataset.split_dataset()

mask_dataset = MaskDataset(train_dir,transform_train)
mask_train, mask_val = age_dataset.split_dataset()

# --dataloader
age_train_loader = torch.utils.data.DataLoader(age_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
age_val_loader = torch.utils.data.DataLoader(age_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

gender_train_loader = torch.utils.data.DataLoader(gender_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
gender_val_loader = torch.utils.data.DataLoader(gender_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

mask_train_loader = torch.utils.data.DataLoader(mask_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
mask_val_loader = torch.utils.data.DataLoader(mask_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# --load models
age_model = models.resnet18(pretrained=True)
gender_model = models.resnet18(pretrained=True)
mask_model = models.resnet18(pretrained=True)

# --initialize models
age_model.fc = torch.nn.Linear(in_features=512, out_features=70, bias=True)
torch.nn.init.kaiming_normal_(age_model.fc.weight)
age_model.fc.bias.data.fill_(0.)

gender_model.fc = torch.nn.Linear(in_features=512, out_features=2, bias=True)
torch.nn.init.kaiming_normal_(gender_model.fc.weight)
gender_model.fc.bias.data.fill_(0.)

mask_model.fc = torch.nn.Linear(in_features=512, out_features=3, bias=True)
torch.nn.init.kaiming_normal_(mask_model.fc.weight)
mask_model.fc.bias.data.fill_(0.);

age_model.to(device)
gender_model.to(device)
mask_model.to(device)

# --loss & metric
loss_fn = torch.nn.CrossEntropyLoss()

LEARNING_RATE = 0.0001
optim_age = torch.optim.Adam(age_model.parameters(), lr=LEARNING_RATE)
optim_gender = torch.optim.Adam(gender_model.parameters(), lr=LEARNING_RATE)
optim_mask = torch.optim.Adam(mask_model.parameters(), lr=LEARNING_RATE)

# --train setting
NUM_EPOCH = 10
BATCH_SIZE = 64
save_dir = os.path.join('/opt/ml/code/baselinecode_v1/model', '10_ensemble')
log_interval = 300
patience = 2  

In [82]:
if os.path.exists(save_dir):
    print('해당 폴더가 이미 존재합니다.')
else:
    os.makedirs(save_dir)
    print('폴더 생성 완료')

해당 폴더가 이미 존재합니다.


### age

In [ ]:
best_val_acc = 0
best_val_loss = np.inf
counter = 0
for epoch in range(NUM_EPOCH):
    # train loop
    age_model.train()
    loss_value = 0
    matches = 0
    cutmix_prob = 0.5  # cutmix 확률을 조절합니다. 0으로 하면, 실행하지 않습니다.
    print('[train loop start !]')
    for idx, train_batch in enumerate(age_train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim_age.zero_grad()

        ## cutmix part start ## 
        r = np.random.rand(1)
        if np.random.rand(1) < cutmix_prob:
            lam = np.random.beta(1.0, 1.0)  # 베타분포는 알파베타가 1이면, uniform분포가 됨
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            label_a = labels
            label_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2, lam = rand_bbox(inputs.size(), lam) 
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            outs = age_model(inputs)
            loss = lam * loss_fn(outs, label_a) + (1. - lam) * loss_fn(outs, label_b)  # mix_target
            preds = torch.argmax(outs, dim=-1)

        ## cutmix part done ##
        else:
            outs = age_model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = loss_fn(outs, labels)

        loss.backward()
        optim_age.step()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % log_interval == 0:
            train_loss = loss_value / log_interval
            train_acc = matches / BATCH_SIZE / log_interval
            current_lr = LEARNING_RATE
            print(
                f"Epoch[{epoch}/{NUM_EPOCH}]({idx + 1}/{len(age_train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0
    
    # val loop
    with torch.no_grad():
        print("[Calculating validation results...]")
        age_model.eval()
        val_loss_items = []
        val_acc_items = []
        figure = None
        for val_batch in age_val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = age_model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = loss_fn(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(age_val_loader)
        val_acc = np.sum(val_acc_items) / len(age_val)
        best_val_loss = min(best_val_loss, val_loss)
        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(age_model.state_dict(), f"{save_dir}/age_best.pth")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # early stopping
        if counter > patience:
            print('Early Stopping...')
            break

        torch.save(age_model.state_dict(), f"{save_dir}/age_last.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        print()

[train loop start !]


### gender

In [ ]:
best_val_acc = 0.
best_val_loss = np.inf
counter = 0
for epoch in range(NUM_EPOCH):
    # train loop
    gemder_model.train()
    loss_value = 0
    matches = 0
    cutmix_prob = 0.5  # cutmix 확률을 조절합니다. 0으로 하면, 실행하지 않습니다.
    print('[train loop start !]')
    for idx, train_batch in enumerate(gender_train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim_gender.zero_grad()

        ## cutmix part start ## 
        r = np.random.rand(1)
        if np.random.rand(1) < cutmix_prob:
            lam = np.random.beta(1.0, 1.0)  # 베타분포는 알파베타가 1이면, uniform분포가 됨
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            label_a = labels
            label_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2, lam = rand_bbox(inputs.size(), lam) 
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            outs = gender_model(inputs)
            loss = lam * loss_fn(outs, label_a) + (1. - lam) * loss_fn(outs, label_b)  # mix_target
            preds = torch.argmax(outs, dim=-1)

        ## cutmix part done ##
        else:
            outs = gender_model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = loss_fn(outs, labels)

        loss.backward()
        optim_gender.step()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % log_interval == 0:
            train_loss = loss_value / log_interval
            train_acc = matches / BATCH_SIZE / log_interval
            current_lr = LEARNING_RATE
            print(
                f"Epoch[{epoch}/{NUM_EPOCH}]({idx + 1}/{len(gender_train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0
        
    # val loop
    with torch.no_grad():
        print("[Calculating validation results...]")
        gender_model.eval()
        val_loss_items = []
        val_acc_items = []
        figure = None
        for val_batch in gender_val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = gender_model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = loss_fn(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(gender_val_loader)
        val_acc = np.sum(val_acc_items) / len(gender_val)
        best_val_loss = min(best_val_loss, val_loss)
        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(gender_model.state_dict(), f"{save_dir}/gender_best.pth")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # early stopping
        if counter > patience:
            print('Early Stopping...')
            break

        torch.save(gender_model.state_dict(), f"{save_dir}/gender_last.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        print()
        print('Calculating validation done !')
        print()

### mask

In [ ]:
best_val_acc = 0
best_val_loss = np.inf
counter = 0
for epoch in range(NUM_EPOCH):
    # train loop
    gemder_model.train()
    loss_value = 0
    matches = 0
    cutmix_prob = 0.5  # cutmix 확률을 조절합니다. 0으로 하면, 실행하지 않습니다.
    print('[train loop start !]')
    for idx, train_batch in enumerate(mask_train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim_mask.zero_grad()

        ## cutmix part start ## 
        r = np.random.rand(1)
        if np.random.rand(1) < cutmix_prob:
            lam = np.random.beta(1.0, 1.0)  # 베타분포는 알파베타가 1이면, uniform분포가 됨
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            label_a = labels
            label_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2, lam = rand_bbox(inputs.size(), lam) 
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            outs = mask_model(inputs)
            loss = lam * loss_fn(outs, label_a) + (1. - lam) * loss_fn(outs, label_b)  # mix_target
            preds = torch.argmax(outs, dim=-1)

        ## cutmix part done ##
        else:
            outs = mask_model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = loss_fn(outs, labels)

        loss.backward()
        optim_mask.step()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % log_interval == 0:
            train_loss = loss_value / log_interval
            train_acc = matches / BATCH_SIZE / log_interval
            current_lr = LEARNING_RATE
            print(
                f"Epoch[{epoch}/{NUM_EPOCH}]({idx + 1}/{len(mask_train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0
        
    # val loop
    with torch.no_grad():
        print("[Calculating validation results...]")
        mask_model.eval()
        val_loss_items = []
        val_acc_items = []
        figure = None
        for val_batch in mask_val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = mask_model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = loss_fn(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(mask_val_loader)
        val_acc = np.sum(val_acc_items) / len(mask_val)
        best_val_loss = min(best_val_loss, val_loss)
        if val_acc > best_val_acc:
            print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            torch.save(mask_model.state_dict(), f"{save_dir}/mask_best.pth")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # early stopping
        if counter > patience:
            print('Early Stopping...')
            break

        torch.save(mask_model.state_dict(), f"{save_dir}/mask_last.pth")
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        print()
        print('Calculating validation done !')
        print()

## 3.Inference

### test dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

### inference

In [ ]:
test_dir = '/opt/ml/input/data/eval'

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, transform_test)

loader = DataLoader(
    dataset,
    shuffle=False
)

# model
age_model.eval()
gender_model.eval()
mask_model.eval()


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
preds_age = []
preds_gender = []
preds_mask = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = age_model(images)
        pred = pred.argmax(dim=-1)
        preds_age.extend(pred.cpu().numpy())

        pred = preds_gender(images)
        pred = pred.argmax(dim=-1)
        preds_gender.extend(pred.cpu().numpy())

        pred = preds_mask(images)
        pred = pred.argmax(dim=-1)
        preds_mask.extend(pred.cpu().numpy())

values = list(zip(preds_age,preds_gender,preds_mask))
answers = []
for value in values:
    print(value)
    age, gender, mask = value
    if age < 30 :
            c = 0
    elif 30 <= age < 60:
        c = 1
    else:
        c = 2
        
    if gender == 1:
        c += 3
    if mask == 1:
        c += 6
    elif mask == 2:
        c += 12
    #print(f'age : {age}, gender: {gender}, mask : {mask}, class : {c}')
    answers.append(c)
submission['ans'] = answers

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(''/opt/ml/code/baselinecode_v1/output', 'output.csv'), index=False)
print('test inference is done!')